In [1]:
%matplotlib qt
import os
import pickle
from sdypy import EMA
import numpy as np
import matplotlib.pyplot as plt 
from EMA_functions import *
import pandas as pd
import matplotlib.animation as animation
import ast
from matplotlib.patches import Ellipse
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.colors as col

c:\Users\thijsmas\Documents\GitHub\pyidi - local\pyidi\pyidi.py:24: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('lk_scipy2', LucasKanadeSc2)
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


TypeError: 'tuple' object is not callable

In [2]:
df = pd.read_csv('H:/My Drive/PHD/HSC/file_descriptions_wlocs.csv')
file_name_basis = 'Full_web_ecc2_new2_Floc18_v'
v_vec = [0, 1, 2, 3, 4, 5, 6, 7, 9]
file_name_vec = [file_name_basis + str(x) for x in v_vec]
file_name_vec_video = [x + '_S01.cihx' for x in file_name_vec]
df_filtered = df[df['filename'].isin(file_name_vec_video)]

roi_size = (9, 9)
reference_image = (0, 30)
df_filtered

,filename,path,ecc,loc,mass,taut/loose,impact/pluck,version,n_frames,fps,Oversatured pixels,Undersatured pixels,prey_ij,spider_ij
118,Full_web_ecc2_new2_Floc18_v0_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,0,8000,8000.0,59,18950,"(358, 231)","(553, 225)"
119,Full_web_ecc2_new2_Floc18_v1_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,1,8000,8000.0,60,19165,"(358, 231)","(553, 225)"
120,Full_web_ecc2_new2_Floc18_v2_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,2,8000,8000.0,498,14590,"(358, 231)","(553, 225)"
121,Full_web_ecc2_new2_Floc18_v3_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,3,8000,8000.0,354,14656,"(358, 231)","(553, 225)"
122,Full_web_ecc2_new2_Floc18_v4_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,4,8000,8000.0,609,16943,"(356, 234)","(553, 225)"
123,Full_web_ecc2_new2_Floc18_v5_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,5,8000,8000.0,727,18127,"(356, 234)","(553, 229)"
124,Full_web_ecc2_new2_Floc18_v6_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,6,8000,8000.0,354,17927,"(356, 237)","(553, 229)"
125,Full_web_ecc2_new2_Floc18_v7_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,7,8000,8000.0,210,18900,"(357, 235)","(553, 229)"
126,Full_web_ecc2_new2_Floc18_v9_S01.cihx,H:\My Drive\PHD\HSC\ecc2\web1\Full_web_ecc2_ne...,2,18,True,Loose,Impact,9,8000,8000.0,150,17438,"(357, 235)","(553, 229)"


In [3]:
# spider_ij_new = [(534, 277), (525, 206), (507, 223)]
# spider_ij_new = [(534, 277), (534, 277), (525, 206), (525, 206), (507, 223), (507, 223)]

n_modes = 8
fn = np.zeros((len(file_name_vec), n_modes))
xi = np.zeros((len(file_name_vec), n_modes))
# color_vec = ['r', 'r', 'b', 'b', 'k', 'k']
# Generate a list of 8 different colors
cmap = plt.cm.get_cmap('seismic')
norm = col.Normalize(vmin=0, vmax=5)
sm = cm.ScalarMappable(norm=norm, cmap=cmap)

ls_vec = ['-', '-', '-', '-', '-', '-', '-', '-', '-']
fig, ax = plt.subplots()
ax.set_xlim([0, 40])
ax.set_xlim([0, 40])
for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    row_index = row.index
    # df_filtered.loc[row_index, 'spider_ij'] = str(spider_ij_new[i])
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    video = EMA_structure.open_video()
    # EMA_structure.open_impact_data()
    # force_signal = EMA_structure.force
    # force_signal = np.append(force_signal, np.zeros(len(force_signal)))
    # n_f = len(force_signal)
    # fs_force = 1/(EMA_structure.t_force[1] - EMA_structure.t_force[0])
    # freq_force = np.fft.rfftfreq(n_f, 1/fs_force)
    # df_force = freq_force[1] - freq_force[0]
    # Force = np.fft.rfft(force_signal, n=n_f) / n_f
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    fn[i, :] = EMA_structure.cam.nat_freq[:n_modes]
    xi[i, :] = EMA_structure.cam.nat_xi[:n_modes]
    # fig, ax = plot_FRF(EMA_structure.cam, fig, ax, c=sm.to_rgba(EMA_structure.cam.F_peak), ls=ls_vec[i], annotate=False, label=f"{EMA_structure.cam.F_peak:.2f}N - Ecc {ecc}")
    fig, ax = plot_H(EMA_structure.cam, fig = fig, ax = ax, c = sm.to_rgba(EMA_structure.cam.F_peak), ls = '-', label = f"{EMA_structure.cam.F_peak:.2f}N", annotate=True)
    print(file_name)
    print(EMA_structure.cam.F_peak)

# Add colorbar
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('F Peak')

plt.show()


fig.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'variable_input_force_FRFs.png'))



C:\Users\thijsmas\AppData\Local\Temp\ipykernel_12052\3265814862.py:9: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = plt.cm.get_cmap('seismic')


H:/My Drive/PHD/HSC\ecc2\web1\Full_web_ecc2_new2_Floc18_v0_S01\Full_web_ecc2_new2_Floc18_v0_S01.cihx
Full_web_ecc2_new2_Floc18_v0
2.4384382064197525


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


H:/My Drive/PHD/HSC\ecc2\web1\Full_web_ecc2_new2_Floc18_v1_S01\Full_web_ecc2_new2_Floc18_v1_S01.cihx
Full_web_ecc2_new2_Floc18_v1
1.7075107219753085


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


...

c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


H:/My Drive/PHD/HSC\ecc2\web1\Full_web_ecc2_new2_Floc18_v7_S01\Full_web_ecc2_new2_Floc18_v7_S01.cihx
Full_web_ecc2_new2_Floc18_v7
0.670057966419753


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


H:/My Drive/PHD/HSC\ecc2\web1\Full_web_ecc2_new2_Floc18_v9_S01\Full_web_ecc2_new2_Floc18_v9_S01.cihx
Full_web_ecc2_new2_Floc18_v9
2.7578766804938266


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\pyMRAW.py:103: UserWarning: Clipped footage! (Total frame: 8000, Original total frame: 87371)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))


In [5]:
# fig, ax = plt.subplots(3, n_modes, sharex=True, sharey=True, subplot_kw={'projection': '3d'})
# fig.tight_layout()

# cmap = plt.cm.get_cmap('seismic')
for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    # video = EMA_structure.open_video()
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    fig, ax = plot_MAC(EMA_structure.cam, 12)
    ax.set_title('Force: ' + str(round(EMA_structure.cam.F_peak, 1)) + 'N')
    fig.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison','MAC', 'MAC_input_force' + str(round(EMA_structure.cam.F_peak, 1)) + '.png'))
    # idx = np.where(EMA_structure.tp[EMA_structure.valid_tps] == EMA_structure.nearest_nut_index)[0]
    # for j in range(n_modes):
    #     Z = plot_mode_shape(EMA_structure.cam, j, EMA_structure.tp, 0, find_Z=True)
    #     ax[ecc, j].scatter(EMA_structure.tp[EMA_structure.valid_tps][:, 1], -EMA_structure.tp[EMA_structure.valid_tps][:, 0], Z, c=Z, cmap=cmap, s=5, alpha=0.6)
    #     if ecc == 0:
    #         ax[ecc, j].set_title('Mode ' + str(j+1))
    #     if j == 0:
    #         ax[ecc, j].set_ylabel('Ecc '+ str(ecc))
            
    #     # ax[ecc, j].set_title('Ecc '+ str(ecc) + ' & Mode ' + str(j+1))

        


c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:71: ComplexWarning: Casting complex values to real discards the imaginary part
  MAC[i, j] = MAC[i, j]/\
c:\Users\thijsmas\AppData\Local\anaconda3\Lib\site-packages\sdypy\EMA\tools.py:7

In [ ]:
n_modes = 5
fig, ax = plt.subplots(3, n_modes, sharex=True, sharey=True, figsize=(18, 8))
fig.tight_layout()
cmap = plt.cm.get_cmap('seismic')
for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    # video = EMA_structure.open_video()
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    plot_MAC(EMA_structure.cam, 6)
    idx = np.where(EMA_structure.tp[EMA_structure.valid_tps] == EMA_structure.nearest_nut_index)[0]
    for j in range(n_modes):
        Z = plot_mode_shape(EMA_structure.cam, j, EMA_structure.tp, 0, find_Z=True)
        ax[ecc, j].scatter(EMA_structure.tp[EMA_structure.valid_tps][:, 1], -EMA_structure.tp[EMA_structure.valid_tps][:, 0], c=Z, cmap=cmap, s=5, alpha=0.6)
        ax[ecc, j].set_aspect('equal')
        if ecc == 0:
            ax[ecc, j].set_title('Mode ' + str(j+1))
        if j == 0:
            ax[ecc, j].set_ylabel('Ecc '+ str(ecc))
        ax[ecc, j].set_xlabel(str(np.round(EMA_structure.cam.nat_freq[j], 2)) + ' Hz')
        ax[ecc, j].set_xticks([])
        ax[ecc, j].set_yticks([])

In [ ]:
n_modes = 6
fig, ax = plt.subplots(3, n_modes, figsize=(18, 8))
fig2 = plt.figure(figsize=(18, 10))
gs = gridspec.GridSpec(3, 2, width_ratios=[4, 1])
ax2 = [[], [], []]
for i in range(3):
    for j in range(2):
        ax2[i].append(plt.subplot(gs[i, j]))
fig.tight_layout()
plt.show()

fig.tight_layout()
cmap = plt.cm.get_cmap('seismic')
width_pixels = 250
height_pixels = 150
width_elipse = 130
height_elipse = 60

n_sections = 8

cmap_circ = cm.get_cmap('Accent')
norm = col.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(norm = norm, cmap=cmap_circ)

for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    # video = EMA_structure.open_video()
    # EMA_structure.open_displacements(roi_size=roi_size, reference_image=reference_image, auto_nut_idx=True)
    EMA_structure.open_cam_EMA()
    idx = np.where(EMA_structure.tp[EMA_structure.valid_tps] == EMA_structure.nearest_nut_index)[0]
    for j in range(n_modes):
        Z = plot_mode_shape(EMA_structure.cam, j, EMA_structure.tp, 0, find_Z=True)
        ax[ecc, j].scatter(EMA_structure.tp[EMA_structure.valid_tps][:, 1], -EMA_structure.tp[EMA_structure.valid_tps][:, 0], c=Z, cmap=cmap, s=5, alpha=0.6)
        if ecc == 0:
            ax[ecc, j].set_title('Mode ' + str(j+1))
        if j == 0:
            ax[ecc, j].set_ylabel('Ecc '+ str(ecc) + f" - {EMA_structure.cam.F_peak:.2f}N")
        ax[ecc, j].set_xlabel(str(np.round(EMA_structure.cam.nat_freq[j], 2)) + ' Hz')
        ax[ecc, j].set_xticks([])
        ax[ecc, j].set_yticks([])
        spider_ij = ast.literal_eval(row['spider_ij'].values[0])
        ax[ecc, j].set_xlim([spider_ij[0]-width_pixels/2, spider_ij[0]+width_pixels/2])
        ax[ecc, j].set_ylim([-spider_ij[1]-height_pixels/2, -spider_ij[1]+height_pixels/2])
        prey_ij = ast.literal_eval(row['prey_ij'].values[0])
        quiver_scale = 1 / np.linalg.norm([prey_ij[0]-spider_ij[0], prey_ij[1]-spider_ij[1]])
        ax[ecc, j].quiver(spider_ij[0], -spider_ij[1], (prey_ij[0]-spider_ij[0])*quiver_scale, -(prey_ij[1]-spider_ij[1])*quiver_scale, angles='xy', scale_units='xy', scale=0.01, color='black')
        # ax[ecc, j].add_patch(Ellipse((spider_ij[0], -spider_ij[1]), width_elipse, height_elipse, edgecolor='red', facecolor='none'))
        
        ellipse_center = (spider_ij[0], -spider_ij[1])
        a = width_elipse / 2
        b = height_elipse / 2

        inside_ellipse = (EMA_structure.tp[:,1] - ellipse_center[0])**2/a**2 + (-EMA_structure.tp[:,0] - ellipse_center[1])**2/b**2 <=1
        inside_and_valid = np.logical_and(inside_ellipse, EMA_structure.valid_tps)
        points_inside_ellipse = EMA_structure.tp[inside_and_valid]
        angles = np.arctan2(-(points_inside_ellipse[:, 0] - spider_ij[1]), points_inside_ellipse[:, 1] - spider_ij[0])
        # ax[ecc, j].scatter(points_inside_ellipse[:, 1], -points_inside_ellipse[:, 0], c=angles, cmap=cmap_circ, s=5)

        frf_all = EMA_structure.cam.frf[inside_ellipse[EMA_structure.valid_tps]]
        
        angle_prey = np.arctan2(-(prey_ij[1] - spider_ij[1]), prey_ij[0] - spider_ij[0])
        if angle_prey < 0:
            angle_prey += 2*np.pi
        angle_prey = angle_prey/(2*np.pi)
        angles[angles < 0] = angles[angles < 0] + 2*np.pi
        normalized_angles = angles/(2*np.pi)
        bucket_edges = np.linspace(-1/(2*n_sections), 1-1/(2*n_sections), num=n_sections + 1) + angle_prey
        bucket_edges = bucket_edges - np.floor(bucket_edges)
        bucket_edges[bucket_edges > 1] -= 1
        bucket_edges[0] = 0
        bucket_edges = np.append(bucket_edges,1)
        bucket_edges = np.sort(bucket_edges)
        angle_sections = (np.digitize(normalized_angles, bucket_edges, right = True)-1).astype(int)
        angle_sections[angle_sections == n_sections] = 0
        
    for freq in EMA_structure.cam.nat_freq[:n_modes]:
        ax2[ecc][0].axvline(x=freq, color='k')
    ax2[ecc][0].set_ylabel(f'$|H|$ - Ecc {ecc}')
    if ecc == 2:
        ax2[ecc][0].set_xlabel('Frequency [Hz]')

    for section in range(8):
        current_points = angle_sections == section
        frf_section = frf_all[current_points]
        if len(frf_section) <= 5:
            lt = ':'
        else:
            lt = '-'
        ax2[ecc][0].semilogy(EMA_structure.cam.freq, np.abs(np.average(frf_section,0)), color=sm.to_rgba(section/(n_sections-1)), linestyle=lt)
        ax2[ecc][1].scatter(points_inside_ellipse[current_points, 1], -points_inside_ellipse[current_points, 0], c=sm.to_rgba(section/(n_sections-1)), s=5)
    ax2[ecc][1].quiver(spider_ij[0], -spider_ij[1], (prey_ij[0]-spider_ij[0])*quiver_scale, -(prey_ij[1]-spider_ij[1])*quiver_scale, angles='xy', scale_units='xy', scale=0.01, color='black')
    ax2[ecc][1].set_xticks([])
    ax2[ecc][1].set_yticks([])
    ax2[ecc][0].set_xlim([0, 35])
    ax2[ecc][1].set_aspect('equal')
    for edge in bucket_edges[1:-1]:
        x_values = [spider_ij[0], spider_ij[0] + 70*np.cos(edge*2*np.pi)]
        y_values = [-spider_ij[1], -spider_ij[1] + 70*np.sin(edge*2*np.pi)]
        ax2[ecc][1].plot(x_values, y_values, color='black', alpha=0.5, linewidth=0.5)
    # ax2[ecc][1].scatter(spider_ij[0] + 10*np.cos(bucket_edges*2*np.pi), -spider_ij[1] + 10*np.sin(bucket_edges*2*np.pi), c='black', s=10)

    # for frf_i, frf in enumerate(frf_all):
    #     ax2[ecc][0].semilogy(EMA_structure.cam.freq, np.abs(frf), color=colors[frf_i], linewidth=0.3)
    #     ax2[ecc][0].set_ylabel(f'$|H|$ - Ecc {ecc}')
    #     if ecc == 2:
    #         ax2[ecc][0].set_xlabel('Frequency [Hz]')
    #     ax2[ecc][0].set_xlim([12, 20])
    #     ax2[ecc][1].scatter(points_inside_ellipse[:, 1], -points_inside_ellipse[:, 0], c=normalized_angles, cmap=cmap_circ, s=5)
    #     ax2[ecc][1].quiver(spider_ij[0], -spider_ij[1], (prey_ij[0]-spider_ij[0])*quiver_scale, -(prey_ij[1]-spider_ij[1])*quiver_scale, angles='xy', scale_units='xy', scale=0.01, color='black')
    #     ax2[ecc][1].set_xticks([])
    #     ax2[ecc][1].set_yticks([])

# Save the figures
fig.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'prey_angle_' + str(int(angle_prey*360)) + f'deg_local_modes_v{v}.png'))
fig2.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'prey_angle_' + str(int(angle_prey*360)) + f'deg_FRFs_v{v}.png'))

for i in range(3):
    for j in range(6):
        ax[i, j].set_xlim([0, 1024])
        ax[i, j].set_ylim([-512, 0])
fig.savefig(os.path.join('G:/.shortcut-targets-by-id/1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx/Illimited Lab Projects/Research Projects/Spiders/Simulations', 'comparison', 'prey_angle_' + str(int(angle_prey*360)) + f'deg_global_modes_v{v}.png'))




In [ ]:
n_modes = 6
fig, ax = plt.subplots(figsize=(18, 8))
fig.tight_layout()
cmap = plt.cm.get_cmap('seismic')

for i, (file_name, file_name_video) in enumerate(zip(file_name_vec, file_name_vec_video)):
    row = df_filtered[df_filtered['filename'] == file_name_video]
    ecc = row['ecc'].values[0]
    EMA_structure = EMA_Structure.load(file_name)
    EMA_structure.open_cam_EMA()


    
    H = EMA_structure.cam.H[inside_and_valid]
